In [55]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
import os

In [56]:
# Ler a tabela
df = pd.read_excel('mochilaCansada.xlsx', index_col=0)
rows, cols = df.shape

# Definir cores fixas para cada CPU
cpu_colors = {
    'A': np.array(mcolors.to_rgb('#4f83cc')),
    'B': np.array(mcolors.to_rgb('#ffcc00')),
    'C': np.array(mcolors.to_rgb('#99cc00')),
    'D': np.array(mcolors.to_rgb('#cc66cc')),
    'E': np.array(mcolors.to_rgb('#ff9933')),
    'F': np.array(mcolors.to_rgb('#ff66b2')),
    'G': np.array(mcolors.to_rgb('#00cccc')),
    'H': np.array(mcolors.to_rgb('#339966')),
}

# Cor para zero
zero_color = np.array(mcolors.to_rgb('#ffcccc'))
# Cor para vazio
empty_color = np.array(mcolors.to_rgb('#d9d9d9'))


In [57]:

# Lista de CPUs
table_cpus = {
    'A': (8, 76),
    'B': (3, 37),
    'C': (6, 47),
    'D': (5, 60),
    'E': (9, 91),
    'F': (2, 21),
    'G': (7, 86),
    'H': (4, 36),
}

cpu_list = list(table_cpus.items())


In [58]:

# Função para reconstruir as CPUs usadas
def get_used_cpus(i, w, dp):
    used = []
    while i > 0 and w >= 0:
        cpu_name, (cost, perf) = cpu_list[i-1]
        if cost <= w and not np.isnan(dp[i-1][w-cost]) and dp[i][w] == perf + dp[i-1][w-cost]:
            used.append(cpu_name)
            w -= cost
            i -= 1
        else:
            i -= 1
    return used[::-1]

# Criar pasta para frames
os.makedirs('simu', exist_ok=True)

# Inicializar matriz vazia (tudo cinza e sem texto)
current_matrix = np.full((rows, cols), np.nan)
frame_num = 0

# Gerar os frames preenchendo célula por célula
for i in range(rows):
    for j in range(cols):
        fig, ax = plt.subplots(figsize=(14, 7))
        ax.axis('off')

        # Atualizar apenas uma célula neste frame
        current_matrix_copy = np.copy(current_matrix)
        current_matrix_copy[i, j] = df.values[i, j]

        # Desenhar cabeçalhos fixos
        ax.text(0.5, rows + 0.5, 'i/w', ha='center', va='center', fontsize=8, fontweight='bold')
        for c in range(cols):
            ax.text(c + 1.5, rows + 0.5, str(c), ha='center', va='center', fontsize=8, fontweight='bold')
        for r in range(rows):
            if r == 0:
                label = '0'
                color = empty_color
            else:
                label = f'{r} ({cpu_list[r-1][0]})'
                color = cpu_colors[cpu_list[r-1][0]]
            ax.add_patch(plt.Rectangle((0, rows - 1 - r), 1, 1, color=color, ec='white'))
            ax.text(0.5, rows - r - 0.5, label, ha='center', va='center', fontsize=8, fontweight='bold', color='black')

        # Preencher células
        for r in range(rows):
            for c in range(cols):
                val = current_matrix_copy[r, c]
                if np.isnan(val):
                    color = empty_color
                    text = ''
                elif val == 0:
                    color = zero_color
                    text = '0'
                else:
                    used = get_used_cpus(r, c, current_matrix_copy)
                    if used:
                        colors = [cpu_colors[cpu] for cpu in used]
                        color = np.mean(colors, axis=0)
                    else:
                        color = zero_color
                    text = str(int(val))

                ax.add_patch(plt.Rectangle((c + 1, rows - 1 - r), 1, 1, color=color, ec='white'))
                if text != '':
                    ax.text(c + 1.5, rows - r - 0.5, text, ha='center', va='center', fontsize=8, color='black')

        ax.set_xlim(0, cols + 1)
        ax.set_ylim(0, rows + 1)

        frame_path = f'simu/frame_{frame_num:03d}.png'
        plt.savefig(frame_path, bbox_inches='tight', pad_inches=0.1)
        plt.close()

        frame_num += 1

        # Atualizar a matriz para o próximo frame
        current_matrix[i, j] = df.values[i, j]

'simu'

'simu'

In [59]:
import imageio
import os

# Pasta onde estão os frames
frames_dir = 'simu'
output_gif = 'mnt/output.gif'  # Salvar o GIF na pasta 'simu'

# Ordenar os frames
frame_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')])

# Carregar os frames
frames = [imageio.imread(frame) for frame in frame_files]

# Criar o GIF
imageio.mimsave(output_gif, frames, duration=0.15)  # 0.15s por frame (ajustável)

output_gif

/var/folders/1g/ltkt__ks3939079g0t1s921h0000gn/T/ipykernel_80762/2503027876.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames = [imageio.imread(frame) for frame in frame_files]


'mnt/output.gif'